In [31]:
import LCMCore
reload("LCMCore")
module comms

import LCMCore: encode, decode

immutable CommsT
    utime::Int64
    format::String
    format_version_major::Int32
    format_version_minor::Int32
    data::Vector{UInt8}
end

function encode(io::IO, comms::CommsT)
    const signature = UInt8[211, 104, 224, 63, 51, 197, 104, 190]
    write(io, signature)
    write(io, hton(comms.utime))
    write(io, hton(UInt32(length(comms.format) + 1)))
    write(io, comms.format)
    write(io, UInt8(0))
    write(io, hton(comms.format_version_major))
    write(io, hton(comms.format_version_minor))
    write(io, hton(Int32(length(comms.data))))
    write(io, hton.(comms.data))
end

function decode(io::IO, ::Type{CommsT})
    signature = read(io, 8)
    @assert signature == UInt8[211, 104, 224, 63, 51, 197, 104, 190]
    utime = Int64(ntoh(read(io, Int64)))
    formatlen = UInt32(ntoh(read(io, UInt32)))
    format = String(read(io, formatlen - 1))
    read(io, 1) # strip off null
    ver_maj = Int32(ntoh(read(io, Int32)))
    ver_min = Int32(ntoh(read(io, Int32)))
    num_bytes = Int32(ntoh(read(io, Int32)))
    data = String(read(io, UInt8, num_bytes))
    CommsT(utime, format, ver_maj, ver_min, data)
end

end

comms

In [32]:
lcm = LCMCore.LCM()

LCMCore.LCM(Ptr{Void} @0x00007fddb8235c90,RawFD(83))

In [33]:
function onmsg(channel, msg)
    @show channel msg
end

onmsg (generic function with 1 method)

In [34]:
LCMCore.subscribe(lcm, "HELLO", onmsg, comms.CommsT)

Ptr{Void} @0x00007fddb8165190

In [35]:
LCMCore.handle(lcm)

info = LCMCore.SubscriptionInfo{comms.CommsT,#onmsg}(comms.CommsT,onmsg)
channel = "HELLO"
msg = comms.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])


true